In [807]:
import redis
from pymongo import MongoClient, ASCENDING
from dateutil.parser import parse
from datetime import datetime, timedelta
import pprint
import os
from termcolor import colored
import warnings
warnings.filterwarnings('ignore')

Hi, welcome to the Mongo-Redis assignment.
For some reason I could not activate the Jupyter notebook via VPN (neither with tunneling to localhost or on the server itself).
Therefore, I installed redis and mongo locally instead (Redis server v=5.0.4, MongoDB shell version v4.0.9)


In [774]:
# Runs redis-server in the background. This phase is not needed when connecting from university.
os.system("nohup redis-server &>/dev/null &")

0

In [772]:
# Create redis client
r = redis.Redis(host='localhost', port=6379, db=0)

In [775]:
# Create Mongo client, When connecting from university just use client=MongoClient()
client=MongoClient(host='localhost', username='mongo',password='mongo') 

In [776]:
client.list_database_names()

['admin', 'config', 'local']

In [777]:
db = client['admin']

In [778]:
db.list_collection_names()

['system.users', 'test', 'jobs', 'system.version']

An important note about collections (and databases) in MongoDB is that they are created lazily - 
none of the above commands have actually performed any operations on the MongoDB server. 
Collections and databases are created when the first document is inserted into them.


In [781]:
collection=db['jobs']

In [782]:
# Create jobID index for speeding up queries and also make sure this value is unique per comapny. 
# The partialFilterExpression helps us to ignore duplicate keys of job_id=NULL
collection.create_index([('jobList.jobId', ASCENDING)],unique=True, 
                        partialFilterExpression= {'jobList.jobId': {'$type': "string"}} ) 



'jobList.jobId_1'

In [783]:
def clear_collection():
    collection.drop()

In [784]:
def clear_cache():
    r.flushdb()

In [785]:
def list_collection():
    """prints the entire content of persistent data in MongoDB"""

    for item in collection.find():
        pprint.pprint(item) 

In [786]:
def list_cache():
    """prints the entire content of the cache"""
    
    for k in r.scan_iter():
        if r.type(k)==b'zset':
            print('This is a sorted Array. The key is {} the value is {}'.format(k, r.zrevrange(k,0,-1)))
        if r.type(k)==b'string':
            print('This is a regular key. The key is {} the value is {}'.format(k, r.get(k)))

In [787]:
def add_company(company):
    """ Adds company to collection if not exists"""
    
    response=collection.update({"companyName": company["companyName"]}, company, upsert=True)


In [788]:
def add_job(job, company_name):
    """ Adds job to joblist of company """

    #Check if company name exists in the collection if not raise 'exception' and return
    response=collection.find_one({"companyName": company_name})
    if response is None:
        print ('Invalid Company Name: {}'.format(company_name))
        print ('Valid Company Names are ')
        print([x['companyName'] for x in collection.find({"Entity": "Company"})])
        return
    #hold incrementing job ids per comapny in cache 
    r.incr(company_name,1) if r.exists(company_name) else r.set(company_name,1)
    job['jobId']='{}_{}'.format(company_name,int(r.get(company_name)))
    # update collection with the new job list for comapny
    collection.update({"companyName": company_name},  {"$push": {"jobsList": job}})
    

In [789]:
def application(candidate, application_time, job_id):
    """ Adds candidate to candidatelist of jobs. 
        ***Please mind that the default string for application time is YYYY-DD-MM **** """
    
    #Check if candidate email exists in current job_id
    response=collection.find_one({"jobsList": {"$elemMatch": {"jobId": job_id, 'candidateList.Email': candidate['Email']}}})
    if response:
        print('{} Has already registered for this position'.format(candidate['Email']))
        return
    #Check if application_time is a valid date
    try:
        updated_at=parse(application_time)
    except ValueError:
        updated_at=datetime.now()
    candidate['updatedAt']=updated_at
    # update collection with the new candidae candidate list for the job
    collection.update({"jobsList": {"$elemMatch": {"jobId": job_id}}}, {"$addToSet": {"jobsList.$.candidateList": candidate}}, upsert=False)
    # update cache, applications per candidate
    current_company=collection.find_one({"jobsList": {"$elemMatch": {"jobId": job_id}}}, {"_id":0, "companyName":1})['companyName']
    r.zadd(candidate['Email'], {current_company:updated_at.timestamp()}) 


In [790]:
def show_latest_apply(k, candidate_email):
    """  Print the k ordered latest company the candidate applied to. If the candidate applied to less than k companies returen all of the companies"""

    assert k>=1 and type(k)==int, "Input k>=1 and type(k)==int"
    result=r.zrevrange(candidate_email,0,k-1)
    if result==[]:
        print('No such candidate {}'.format(candidate_email))
        return
    print('The latest companies that the candidate {} applied to'.format(candidate_email))
    print(r.zrevrange(candidate_email,0,k-1))

In [791]:
def show_candidates(job_id, skills):
    """Print candidates’ emails that applied for a given position and have the specified skills."""
    
    response=collection.aggregate([
        {'$match':   {"jobsList": {"$elemMatch": {"jobId": job_id}} , 
                    'jobsList.candidateList.skills': {"$all":  skills}}}, #match job_id and requiered skills (reduces number of computations)
        {'$project': {"jobsList":  1, "_id":0}}, #select joblist
        {'$unwind': '$jobsList'}, #explode joblist array
        {'$match':   {"jobsList.jobId": job_id}}, #match job_id
        {'$project': {"jobsList.candidateList":  1, "_id":0}}, #select candidate list
        {'$unwind': '$jobsList.candidateList'}, #explode candidate list array
        {'$match':   {'jobsList.candidateList.skills': {"$all":  skills}}}, #match candidates that have all skills requiered
        {'$project': {"jobsList.candidateList.Email":  1, "_id":0}} #select email
                                  ])
    print('List of Candidates that have the requiered skills for the job')
    for item in response:
        pprint.pprint(item['jobsList']['candidateList']['Email'])
    


      

In [792]:
def count_jobs_by_company():
    """ counts how many jobs have been posted for each company name """

    for k in r.scan_iter(): #We simply need to retrieve from cache
        if r.type(k)==b'string': #check that the key is from type string and not Set for example
            print('Company Name: {}. Number of jobs: {}'.format(k.decode('utf-8'), int(r.get(k))))

In [793]:
def count_candidates_by_job():
    """counts how many candidates have for each Job ID during the last 30 days"""
    
    filter_date=datetime.now() + timedelta(-30) # Get date 30 days ago
    result=collection.aggregate([ {'$match': {'jobsList.candidateList.updatedAt': {'$gte': filter_date} }}, #match candidates who submitted for a job in the past 30 days (reduces number of computations)
                              {"$unwind": "$jobsList"},  #explode job array
                              {'$project': {"jobsList":  1,  "_id":0}}, #select joblist only
                              {"$unwind": "$jobsList.candidateList"}, #explode candidate list
                              {'$project': {"jobsList.candidateList":  1, "jobsList.jobId":1, "_id":0}}, #select job_id and candidate list only
                              {'$match': {'jobsList.candidateList.updatedAt': {'$gte': filter_date} }}, #match candidates who submitted for a job in the past 30 days
                              {"$group": {"_id": "$jobsList.jobId", 'numOfCandidatesLastMonth': { '$sum': 1 }}} #group by jobId, count number of candidates
                             
                            ])
    for item in result:
        pprint.pprint(item)
    
    
    

In [794]:
def recovery():
    """ should be executed whenever the Redis server restarts (for example
    after a server crash like power failure). This function will use the persistent data (stored on MongoDB) to
    reload data to the cache (Redis) """
    
    # Aggregate number of jobs per company from persistant data 
    result=collection.aggregate([{"$unwind": "$jobsList"}, 
                          {"$group": {"_id": "$companyName", 'numOfJobs': { '$sum': 1 }}}])
    clear_cache() #makes sure cache is clear
    for item in result: #push to cache
        r.set(item['_id'], item['numOfJobs'])
    # Aggregate Company per candidate sorted by applicatino date
    result=collection.aggregate([
                        {"$unwind": "$jobsList"}, #explode job array
                        {"$unwind": "$jobsList.candidateList"}, #explode candidate array
                        {'$project': {'companyName':1 ,'jobsList.candidateList.Email':1, 'jobsList.candidateList.updatedAt':1,'_id':0}},
                        {'$group': {"_id": {'candidate': '$jobsList.candidateList.Email', 'company': '$companyName'},
                                   'min_date': {'$min': '$jobsList.candidateList.updatedAt'}}}, #group by candidate, company , aggregate minimum application date
                        {'$sort': {'_id':1, 'min_date':1}}]) #sort in asceding order
    
    for item in result: #push to cache
        r.zadd(item['_id']['candidate'], {item['_id']['company']:item['min_date'].timestamp()})
    
    

In [820]:
def execute():
    """calls all functions with the arguments that are given in the examples"""
    
    clear_cache()
    clear_collection()
    add_company({'companyName':'TAU', 'companyDescription':'University'})
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    print(colored(' We expect to see the new company in the db', 'cyan', 'on_yellow', ['bold']))
    print('\n'*3)
    list_collection()
    print('\n'*2)
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    add_job({'jobName':'bi developer', 'publishDate': 'Tel Aviv', 'skills':['python','big data','mongodb'],
             'Status':'open','publishDate':'01-02-2019'},'TAU')
    print(colored(' We expect to see the bi_developer job added to TAU', 'cyan', 'on_yellow', ['bold']))
    print('\n'*3)
    list_collection()
    print('\n'*2)
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    application({'candidateName':'laura', 'Email':'laura@gmail.com', 
                 'linkedin':'https://www.linkedin.com/in/laura/', 'skills': ['python','sql']}, '01-02-2019 15:00:00', 'TAU_1')
    print(colored(' We expect to see laura added to the candidate list', 'cyan', 'on_yellow', ['bold']))
    print('\n'*3)
    list_collection()
    print('\n'*2)
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    print(colored(' We expect to see that laura only applied to TAU', 'cyan', 'on_yellow', ['bold']))
    print('\n'*3)
    show_latest_apply(10, 'laura@gmail.com')
    print('\n'*2)
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    print(colored(' We expect to see that laura has the skills', 'cyan', 'on_yellow', ['bold']))
    print('\n'*3)
    show_candidates('TAU_1', ['python','sql'])
    print('\n'*2)
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    print(colored(' We expect to see that TAU has one job', 'cyan', 'on_yellow', ['bold']))
    print('\n'*3)
    count_jobs_by_company()
    print('\n'*2)
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    print(colored("""Laura's application was more than 30 days ago, so we will not see any candidates per job, But if you
         will change the application date to '17-04-2019' you will see results. just make sure that if you give this
         date '04-03-2019' it means April 3rd not March 4th!""", 'cyan', 'on_yellow', ['bold']))
    print('\n'*3)
    count_candidates_by_job()
    print('\n'*2)
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    print(colored(' This is how the cache looks like at the moment', 'cyan', 'on_yellow', ['bold']))
    print('\n'*3)
    list_cache()
    print('\n'*2)
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    print(colored(' Now lets clear it and use the recovery function', 'cyan', 'on_yellow', ['bold']))
    clear_cache()
    recovery()
    print('\n'*3)
    list_cache()
    print(colored('**********************************************', 'cyan', 'on_white', ['blink']))
    print(colored(' Were back to business! Hope you enjoyed', 'cyan', 'on_yellow', ['bold']))

    




In [821]:
execute()

**********************************************
 We expect to see the new company in the db




{'_id': ObjectId('5cb6ff47cea9397f17474e43'),
 'companyDescription': 'University',
 'companyName': 'TAU'}



**********************************************
 We expect to see the bi_developer job added to TAU




{'_id': ObjectId('5cb6ff47cea9397f17474e43'),
 'companyDescription': 'University',
 'companyName': 'TAU',
 'jobsList': [{'Status': 'open',
               'jobId': 'TAU_1',
               'jobName': 'bi developer',
               'publishDate': '01-02-2019',
               'skills': ['python', 'big data', 'mongodb']}]}



**********************************************
 We expect to see laura added to the candidate list




{'_id': ObjectId('5cb6ff47cea9397f17474e43'),
 'companyDescription': 'University',
 'companyName': 'TAU',
 'jobsList': [{'Status': 'open',
               'candidateList': [{'Email': 'laura@gmail.com',
                                  'candidateName': 'laura',
      


## From here and on there are more tests with data that I have made up








In [823]:
#Create Entities
company1={'Entity': 'Company','companyName': 'Avantis', 'companyDescription': 'Adtech', 'jobsList': []}
company2={'Entity': 'Company','companyName': 'Browsi', 'companyDescription': 'Adtech', 'jobsList': []}
job1= { 'Entity': 'Job', 'jobName': 'BI-developer', 'requirementsList':['python','sql'], 'Location':None, 'publishDate':None, 'Status':None, 'candidateList':[] }
job2= { 'Entity': 'Job', 'jobName': 'QA', 'requirementsList':[], 'Location':None, 'publishDate':None, 'Status':None, 'candidateList':[] } 
candidate1={'Entity': 'Candidate', 'candidateName':'laura', 'Email':'laura@gmail.com',
'Linkedin':'https://www.linkedin.com/in/laura/', 'skills': ['python','sql']}
candidate2={'Entity': 'Candidate', 'candidateName':'james', 'Email':'james@hotmail.com',
'Linkedin':'https://www.linkedin.com/in/james/', 'skills': ['python']}
candidate3={'Entity': 'Candidate', 'candidateName':'Andy', 'Email':'andy@hotmail.com',
'Linkedin':'https://www.linkedin.com/in/andy/', 'skills': ['sql']}

In [824]:
clear_cache()

In [825]:
clear_collection() 

### Let's Add some companies 

In [826]:
add_company(company1)
add_company(company2)
add_company(company1) # This will not be added, duplicate company

### We expect to have 2 companies:
1. Browsi
2. Avantis

In [827]:
list_collection() 

{'Entity': 'Company',
 '_id': ObjectId('5cb7005ecea9397f17474e8d'),
 'companyDescription': 'Adtech',
 'companyName': 'Avantis',
 'jobsList': []}
{'Entity': 'Company',
 '_id': ObjectId('5cb7005ecea9397f17474e8f'),
 'companyDescription': 'Adtech',
 'companyName': 'Browsi',
 'jobsList': []}


### Let's Add some jobs

In [828]:
add_job(job1, 'Browsi')
add_job(job2, 'Browsi')
add_job(job1, 'Browsi') # It is possible to have the same job twice (maybe we need two QAs for example)
add_job(job1, 'Avantis')
add_job(job1, 'Hive') #This will not be added, invalid company name

Invalid Company Name: Hive
Valid Company Names are 
['Avantis', 'Browsi']


### We expect to have 4 jobs:
3 for Browsi
1 for Avantis

In [829]:
list_collection() 

{'Entity': 'Company',
 '_id': ObjectId('5cb7005ecea9397f17474e8d'),
 'companyDescription': 'Adtech',
 'companyName': 'Avantis',
 'jobsList': [{'Entity': 'Job',
               'Location': None,
               'Status': None,
               'candidateList': [],
               'jobId': 'Avantis_1',
               'jobName': 'BI-developer',
               'publishDate': None,
               'requirementsList': ['python', 'sql']}]}
{'Entity': 'Company',
 '_id': ObjectId('5cb7005ecea9397f17474e8f'),
 'companyDescription': 'Adtech',
 'companyName': 'Browsi',
 'jobsList': [{'Entity': 'Job',
               'Location': None,
               'Status': None,
               'candidateList': [],
               'jobId': 'Browsi_1',
               'jobName': 'BI-developer',
               'publishDate': None,
               'requirementsList': ['python', 'sql']},
              {'Entity': 'Job',
               'Location': None,
               'Status': None,
               'candidateList': [],
         

### Let's check the cache:
we expect to see:
Avanits-1
Browsi-3

In [830]:
list_cache() 

This is a regular key. The key is b'Avantis' the value is b'1'
This is a regular key. The key is b'Browsi' the value is b'3'


### Let's add some candidates

In [831]:
application(candidate1, '17-02-2019 16:00:00', "Avantis_1")
application(candidate1, '15-04-2019 17:00:00', "Avantis_1") # should not be added, already apllied for job


laura@gmail.com Has already registered for this position


### We expect to see one candidate (Laura) that  is added 

In [832]:
list_collection() # We expect to sone candidate (laura) added to Avantis_1 job

{'Entity': 'Company',
 '_id': ObjectId('5cb7005ecea9397f17474e8d'),
 'companyDescription': 'Adtech',
 'companyName': 'Avantis',
 'jobsList': [{'Entity': 'Job',
               'Location': None,
               'Status': None,
               'candidateList': [{'Email': 'laura@gmail.com',
                                  'Entity': 'Candidate',
                                  'Linkedin': 'https://www.linkedin.com/in/laura/',
                                  'candidateName': 'laura',
                                  'skills': ['python', 'sql'],
                                  'updatedAt': datetime.datetime(2019, 2, 17, 16, 0)}],
               'jobId': 'Avantis_1',
               'jobName': 'BI-developer',
               'publishDate': None,
               'requirementsList': ['python', 'sql']}]}
{'Entity': 'Company',
 '_id': ObjectId('5cb7005ecea9397f17474e8f'),
 'companyDescription': 'Adtech',
 'companyName': 'Browsi',
 'jobsList': [{'Entity': 'Job',
               'Location': None,

### Let's add some more candidates to different positions

In [833]:
application(candidate3, '13-02-2019 18:00:00', "Avantis_1") #andy
application(candidate2, '13-04-2019 18:00:00', "Browsi_2") #james
application(candidate2, '14-04-2019 18:00:00', "Browsi_1") #james
application(candidate1, '16-03-2019 18:00:00', "Browsi_1") #laura

### We expect to see 5 candidates:
Laura, Anday for BI-developer at Avantis ; Laura, James for BI-developer at Browsi ; James for QA at Browsi

In [834]:
list_collection() 

{'Entity': 'Company',
 '_id': ObjectId('5cb7005ecea9397f17474e8d'),
 'companyDescription': 'Adtech',
 'companyName': 'Avantis',
 'jobsList': [{'Entity': 'Job',
               'Location': None,
               'Status': None,
               'candidateList': [{'Email': 'laura@gmail.com',
                                  'Entity': 'Candidate',
                                  'Linkedin': 'https://www.linkedin.com/in/laura/',
                                  'candidateName': 'laura',
                                  'skills': ['python', 'sql'],
                                  'updatedAt': datetime.datetime(2019, 2, 17, 16, 0)},
                                 {'Email': 'andy@hotmail.com',
                                  'Entity': 'Candidate',
                                  'Linkedin': 'https://www.linkedin.com/in/andy/',
                                  'candidateName': 'Andy',
                                  'skills': ['sql'],
                                  'updatedAt': d

### Let's check The cache

### We expect to see Laura--> ['Browsi', 'Avantis'] ; James --> ['Browsi'] ; Andy->['Avantis']

In [835]:
list_cache() 

This is a regular key. The key is b'Avantis' the value is b'1'
This is a sorted Array. The key is b'james@hotmail.com' the value is [b'Browsi']
This is a sorted Array. The key is b'laura@gmail.com' the value is [b'Browsi', b'Avantis']
This is a sorted Array. The key is b'andy@hotmail.com' the value is [b'Avantis']
This is a regular key. The key is b'Browsi' the value is b'3'


### We expect to see [Browsi, Avantis ] for Laura

In [836]:
show_latest_apply(10, candidate1['Email']) 

The latest companies that the candidate laura@gmail.com applied to
[b'Browsi', b'Avantis']


#### We expect to see only Laura since she is the only one that knows SQL from all the Browsi- BI Developer candidates

In [838]:
show_candidates('Browsi_1', ['sql']) 
    

List of Candidates that have the requiered skills for the job
'laura@gmail.com'


#### We expect to see only Laura since she is the only one that knows SQL **AND * * Python from all the Browsi- BI Developer candidates

In [839]:
show_candidates('Browsi_1', ['sql', 'python']) #we expect to see laura


List of Candidates that have the requiered skills for the job
'laura@gmail.com'


#### We expect to see only both Laura and James since they both know Python

In [840]:
show_candidates('Browsi_1', ['python']) #we expect to see laura and james


List of Candidates that have the requiered skills for the job
'james@hotmail.com'
'laura@gmail.com'


#### We expect to see nobody, since there is no candidate that knows Java for this position

In [841]:
show_candidates('Browsi_1', ['python','sql', 'java']) #we expect to see nobody


List of Candidates that have the requiered skills for the job


#### Same thing for these two examples

In [842]:
show_candidates('Browsi_3', ['python']) #we expect to see nobody


List of Candidates that have the requiered skills for the job


In [843]:
show_candidates('Browsi_2', ['sql']) #we expect to see nobody


List of Candidates that have the requiered skills for the job


#### We expect to see only James since he is the only candidate for Browsi-QA position, and he knows Python.

In [844]:
show_candidates('Browsi_2', ['python']) #we expect to see james


List of Candidates that have the requiered skills for the job
'james@hotmail.com'


### Let's count jobs by company

#### We Expect to see 1 job for Avantis, 3 jobs for Browsi

In [845]:
count_jobs_by_company() # We expect to see Avantis :1 Browsi: 3

Company Name: Avantis. Number of jobs: 1
Company Name: Browsi. Number of jobs: 3


### Let's check Candidates by job (only candidates that applied in the last 30 days)

#### We expect to see one candidate for Browsi-BI developer and one candidate for Browsi-QA

#### Applications for Avantis- Bi developer were too old, and also Laura's application for Browsi-BI developer. Leaving us with 2

In [699]:
count_candidates_by_job() #We expect Avantis 1--> 0 Browsi1--> 1 Browsi2--> 1 Browsi3-->0

{'_id': 'Browsi_2', 'numOfCandidatesLastMonth': 1}
{'_id': 'Browsi_1', 'numOfCandidatesLastMonth': 1}


### Let's check our cache again

In [846]:
list_cache()

This is a regular key. The key is b'Avantis' the value is b'1'
This is a sorted Array. The key is b'james@hotmail.com' the value is [b'Browsi']
This is a sorted Array. The key is b'laura@gmail.com' the value is [b'Browsi', b'Avantis']
This is a sorted Array. The key is b'andy@hotmail.com' the value is [b'Avantis']
This is a regular key. The key is b'Browsi' the value is b'3'


#### Let's clear and recover

In [847]:
clear_cache()

In [848]:
recovery()

#### Let's check the cache again

In [849]:
list_cache() # We expcet to see the same cache

This is a regular key. The key is b'Avantis' the value is b'1'
This is a sorted Array. The key is b'james@hotmail.com' the value is [b'Browsi']
This is a sorted Array. The key is b'andy@hotmail.com' the value is [b'Avantis']
This is a sorted Array. The key is b'laura@gmail.com' the value is [b'Browsi', b'Avantis']
This is a regular key. The key is b'Browsi' the value is b'3'


### The cache is back to biz